In [1]:
# Purpose: Preparing ERA5 net surface heat flux fields for plotting
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #                         
#     Maurice F. Huguenin, m.huguenin-virchaux@unsw.edu.au, 21. 12. 2023      #                                                
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #                 
base = '/g/data/e14/mv7494/access-om2/archive/025deg_jra55_iaf/'
save = '/g/data/e14/mv7494/'

import xarray as xr # for working with labelled multi-dimensional arrays                                                                
import numpy as np # for numerical operations                                                           
import cmocean as cm
from datetime import datetime # package for printing time in plot                                                                                                       
import matplotlib.pyplot as plt # for matlab-like plotting                                              
import cartopy.crs as ccrs # for maps     
import itertools
import cosima_cookbook as cc
import pandas as pd
from matplotlib import gridspec
from cdo import *
cdo = Cdo()
import os
import sys
import warnings
import matplotlib.path as mpath
import time
import matplotlib.animation as animation # library that makes the movie
import matplotlib.patheffects as PathEffects
warnings.filterwarnings('ignore') # suppress warnings
from matplotlib import ticker
import nc_time_axis
import cftime
import cartopy.mpl.ticker as cticker
import string # for subplot annotation
import cartopy.feature as cfeature
from matplotlib import gridspec
import bottleneck as bn # for time series moving average
from scipy import stats # create linear trend over the 3rd to 5th cycles for the upper 2000 m

#import warnings
#warnings.filterwarnings('ignore') # suppress warnings

import nc_time_axis
import cftime

# for displaying multiple output in one cell
from IPython.display import display as dp

from dask.distributed import Client
client = Client(n_workers=16)        
client
rotblau = [[0.4 , 0.  , 0.12], [0.47, 0.02, 0.13], [0.62, 0.07, 0.16], [0.66, 0.16, 0.2],
           [0.8 , 0.3 , 0.27], [0.87, 0.44, 0.35], [0.93, 0.58, 0.45], [0.96, 0.7 , 0.58], 
           [0.98, 0.8 , 0.71], [0.98, 0.94, 0.92], [0.93, 0.95, 0.96], [0.85, 0.91, 0.95],
           [0.76, 0.87, 0.93], [0.64, 0.8 , 0.89], [0.49, 0.72, 0.84], [0.34, 0.62, 0.79], 
           [0.23, 0.53, 0.74], [0.16, 0.44, 0.7] , [0.1 ,0.35, 0.6], [0.05, 0.24, 0.45], [0.02, 0.19, 0.38]]

### Calculating daily anomalies for different variables

In [6]:
%%time
session = cc.database.create_session()
import matplotlib.path as mpath 
# -------------------------------------------------------------------------------------------- #
NA_lat        = [    60,    361] # [    60,    361] = 75°N - 0°                                #
NA_lon        = [   320,    801] # [   320,    801] = 100°W - 20°E                             #
time          = [350650, 613633] # [350650, 613633] = 1980-01-01T00:00:00, 2010-01-01T01:00:00 #
# -------------------------------------------------------------------------------------------- #
era_path = '/g/data/rt52/era5/single-levels/reanalysis/'
vars = ['slhf', 'sshf', 'ssrd', 'strd'] 

# Step I:     calculating daily climatology for the variable
for f in range(len(vars)):
    if vars[f] in ['slhf', 'sshf', 'ssrd', 'strd']:
        average_DT = (60*60)
        print('Time integrating variables')
    else:
        average_DT = 1
    if not os.path.isdir('/g/data/e14/mv7494/ERA5/'+vars[f]):
        print('creating folder')
        os.mkdir('/g/data/e14/mv7494/ERA5/'+vars[f])
    filename = '/g/data/e14/mv7494/ERA5/'+vars[f]+'/ERA5_NorthAtlantic_'  

    for year in range(1959,2024): # loop through the climatological 1980-2010 period
        if os.path.exists(filename+vars[f]+'_daily_'+str(year)+'.nc'):
            print(str(year)+' already done')
            continue
        print('Preparing daily '+vars[f]+' for year '+ str(year))
        data = xr.open_mfdataset(era_path + vars[f] + '/'+str(year)+'/*.nc', chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}
                                     )[vars[f]][:,NA_lat[0]:NA_lat[1],NA_lon[0]:NA_lon[1]].resample(time='D').mean() / average_DT
        data = data.to_dataset(name=vars[f]).to_netcdf(filename+vars[f]+'_daily_'+str(year)+'.nc')
    print('--------------------------')
# Step II: now creating daily climatology over 1981-2010
    data_clim       = xr.open_mfdataset(filename+vars[f]+'_daily_*.nc', chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}, decode_times=True)[vars[f]][7670:18629]
    for year in range(1959,2024):
        if os.path.exists(filename+'daily_anomalies_'+str(year)+'.nc'):
            print(str(year)+' already done')
            continue
        print('Preparing daily '+vars[f]+' anomalies for year '+ str(year))

        data            = xr.open_mfdataset(filename+vars[f]+'_daily_'+str(year)+'.nc', chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}, decode_times=True)[vars[f]]
        data_day_anom = (data.groupby('time.dayofyear') - data_clim.groupby('time.dayofyear').mean('time')).to_dataset(name=vars[f]+'_day_anom')
        
        # add attributes
        data_day_anom.attrs = {'Units': '[W/m2]', 
                           'Project': 'North Atlantic Ocean Heat Wave 2023',
                           'Author': 'Maurice F. Huguenin', 
                           'Data set': 'ERA5 reanalysis',
                            'Climatology': '1980-2010 daily climatology',
                           'Created': '2023-12-21', 
                           'Mail': 'm.huguenin-virchaux@unsw.edu.au', 
                           'Funding': 'UNSW (Program code 1476), ARC ACEAS (ARC Grant No. SR200100008), ARC CLEX (ARC Grant No. CE170100023)'}
        data_day_anom.to_netcdf(filename+vars[f]+'_daily_anomalies_'+str(year)+'.nc')
# Wall time: 1min for one year

Time integrating variables
1959 already done
1960 already done
1961 already done
1962 already done
1963 already done
1964 already done
1965 already done
1966 already done
1967 already done
1968 already done
1969 already done
1970 already done
1971 already done
1972 already done
1973 already done
1974 already done
1975 already done
1976 already done
1977 already done
1978 already done
1979 already done
1980 already done
1981 already done
1982 already done
1983 already done
1984 already done
1985 already done
1986 already done
1987 already done
1988 already done
1989 already done
1990 already done
1991 already done
1992 already done
1993 already done
1994 already done
1995 already done
1996 already done
1997 already done
1998 already done
1999 already done
2000 already done
2001 already done
2002 already done
2003 already done
2004 already done
2005 already done
2006 already done
2007 already done
2008 already done
2009 already done
2010 already done
2011 already done
2012 already done
2

2023-12-31 15:47:36,612 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-12-31 15:47:37,425 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2023-12-31 15:47:38,805 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)


Preparing daily slhf anomalies for year 1965


2023-12-31 15:47:56,264 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-12-31 15:47:57,039 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-12-31 15:47:58,202 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-12-31 15:47:59,036 - distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
2023-12-31 15:48:01,206 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-12-31 15:48:01,760 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-12-31 15:48:02,575 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-12-31 15:48:04,126 - distributed.utils_perf - WARNING - full garbage collections took

Preparing daily slhf anomalies for year 1966


...

Preparing daily strd anomalies for year 2022


2023-12-31 17:44:44,976 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2023-12-31 17:44:47,587 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)
2023-12-31 17:44:52,314 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-12-31 17:45:00,216 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


Preparing daily strd anomalies for year 2023


2023-12-31 17:45:15,080 - distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
2023-12-31 17:45:18,550 - distributed.utils_perf - WARNING - full garbage collections took 14% CPU time recently (threshold: 10%)


CPU times: user 1h 43min 52s, sys: 4min 43s, total: 1h 48min 35s
Wall time: 2h 11s


### Calculating net surface heat flux anomalies

In [7]:
%%time
session = cc.database.create_session()
import matplotlib.path as mpath 
# -------------------------------------------------------------------------------------------- #
NA_lat        = [    60,    361] # [    60,    361] = 75°N - 0°                                #
NA_lon        = [   320,    801] # [   320,    801] = 100°W - 20°E                             #
time          = [350650, 613633] # [350650, 613633] = 1980-01-01T00:00:00, 2010-01-01T01:00:00 #
# -------------------------------------------------------------------------------------------- #
filename = '/g/data/e14/mv7494/ERA5/nsf/ERA5_NorthAtlantic_nsf'
era_path = '/g/data/rt52/era5/single-levels/reanalysis/'
vars = ['slhf', 'sshf', 'ssrd', 'strd'] 
#       'slhf'                          # Surface upward latent heat flux (J/m2)
#               'sshf'                  # Surface upward sensible heat flux (J/m2)
#                       'ssrd'          # Surface downwelling short-wave flux (solar) in air (J/m2)
#                              'strd'   # Surface long-wave (thermal) radiation downwards (J/m2)
average_DT = (60*60)
# (UI)
for year in range(1959,2024):
    if os.path.exists(filename+'_daily_'+str(year)+'.nc'):
        print(str(year)+' already done')
        continue
    print('Preparing daily nsf_clim for year '+ str(year))
    era_short = '/'+str(year)+'/*.nc' 
    
    slhf = xr.open_mfdataset(era_path + 'slhf' + era_short, chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}).slhf[:,NA_lat[0]:NA_lat[1],NA_lon[0]:NA_lon[1]].resample(time='D').mean() / average_DT
    sshf = xr.open_mfdataset(era_path + 'sshf' + era_short, chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}).sshf[:,NA_lat[0]:NA_lat[1],NA_lon[0]:NA_lon[1]].resample(time='D').mean() / average_DT
    ssrd = xr.open_mfdataset(era_path + 'ssrd' + era_short, chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}).ssrd[:,NA_lat[0]:NA_lat[1],NA_lon[0]:NA_lon[1]].resample(time='D').mean() / average_DT
    strd = xr.open_mfdataset(era_path + 'strd' + era_short, chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}).strd[:,NA_lat[0]:NA_lat[1],NA_lon[0]:NA_lon[1]].resample(time='D').mean() / average_DT
    nsf = ((slhf + sshf + ssrd + strd)).to_dataset(name=suffix).to_netcdf(filename+'_daily_'+str(year)+'.nc')
print('-----------------')

# (II) now creating daily climatology over all years
nsf_clim       = xr.open_mfdataset(filename+'_daily_*.nc', chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}, decode_times=True).nsf_clim[7670:18629]
for year in range(1959,2024):
    if os.path.exists(filename+'_daily_anomalies_'+str(year)+'.nc'):
        print(str(year)+' already done')
        continue
    print('Preparing daily nsf anomalies for year '+ str(year))
    
    nsf            = xr.open_mfdataset(filename+'_daily_'+str(year)+'.nc', chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}, decode_times=True).nsf_clim
    nsf_day_anom = (nsf.groupby('time.dayofyear') - nsf_clim.groupby('time.dayofyear').mean('time')).to_dataset(name='nsf_day_anom')
    # add attributes
    nsf_day_anom.attrs = {'Units': '[W/m2]', 
                       'Project': 'North Atlantic Ocean Heat Wave 2023',
                       'Author': 'Maurice F. Huguenin', 
                       'Data set': 'ERA5 reanalysis',
                        'Climatology': '1980-2010 daily climatology',
                       'Created': '2023-12-21', 
                       'Mail': 'm.huguenin-virchaux@unsw.edu.au', 
                       'Funding': 'UNSW (Program code 1476), ARC ACEAS (ARC Grant No. SR200100008), ARC CLEX (ARC Grant No. CE170100023)'}
    nsf_day_anom.to_netcdf(filename+'_daily_anomalies_'+str(year)+'.nc')
# Wall time: 1min for one year

1959 already done
1960 already done
1961 already done
1962 already done
1963 already done
1964 already done
1965 already done
1966 already done
1967 already done
1968 already done
1969 already done
1970 already done
1971 already done
1972 already done
1973 already done
1974 already done
1975 already done
1976 already done
1977 already done
1978 already done
1979 already done
1980 already done
1981 already done
1982 already done
1983 already done
1984 already done
1985 already done
1986 already done
1987 already done
1988 already done
1989 already done
1990 already done
1991 already done
1992 already done
1993 already done
1994 already done
1995 already done
1996 already done
1997 already done
1998 already done
1999 already done
2000 already done
2001 already done
2002 already done
2003 already done
2004 already done
2005 already done
2006 already done
2007 already done
2008 already done
2009 already done
2010 already done
2011 already done
2012 already done
2013 already done
2014 alrea